In [12]:
import pandas as pd
import numpy as np
import os
from IPython.display import display

In [ ]:
os.chdir('/mnt/c/Users/amand/Desktop')  # sample.xlsx is on my C drive
df = pd.read_excel('sample.xlsx')
os.chdir('/home/amandalin047/cloze_2023')    # but I'm running Jupyter in Ubuntu 22.04, which is my preferred working directory
display(df)

In [3]:
# This piece checks if "Optimal" and "Unexpected" have overlapping characters

opt, unexp = [set([*i]) for i in df['Optimal']], [set([*i]) for i in df['Unexpected']]

overlap = np.zeros(len(df))
for (i,x), (j,y) in zip(enumerate(opt), enumerate(unexp)):
    for char in x:
        if char in y:
            overlap[i] = 1
            break
            
df_styled = df.style.apply(lambda col: ['color: orange' if overlap[i] > 0 else None for i in range(len(col))],
                           subset='Unexpected')
df_styled.to_excel('Char_Overlap_Checked.xlsx')

In [14]:
# This piece selects the unique best completions provided by offline participants

col = ['Best'+str(i) for i in range(1,21)]
best = set(sum([[df.iloc[i][j] for j in col] for i in df.index], []))

for i in best:
    if pd.isnull(i):
        best.remove(i)
        break
        
pd.DataFrame(best, columns=['All_Best']).to_excel('All_Best.xlsx')

In [15]:
# This piece is for calculating the cloze probability for each best completion provided by offline participants
# and is basically just Excel in Python form

def MATCH(lookup_values, lookup_array , match_type=1):
    '''Analogous to the Excel MATCH function'''
    pos = []
    if match_type == 1:
        ascend_array = np.sort(lookup_array)
        for i in lookup_values:
            for j, x in enumerate(ascend_array):
                if x > i:
                    pos.append(lookup_array.tolist().index(ascend_array[j]))
                    break
                    
    elif match_type == 0:
        for i in lookup_values:
            for j in range(len(lookup_array)):
                if i == lookup_array[j]:
                    pos.append(j)
                    break
                    
    elif match_type == -1:
        descend_array = np.sort(lookup_array)[::-1]
        for i in lookup_values:
            for j, x in enumerate(descend_array):
                if x < i:
                    pos.append(lookup_array.tolist().index(descend_array[j]))       
    else:
        print('match_type takes values 1, 0, or -1')
            
    if pos == []: return '#N/A'
    else: return pos
    
    
def MODE(lookup_array):
    '''Analogous to the Excel MODE function'''
    occur = []
    for i in lookup_array:
        o = 0
        for j in lookup_array:
            if i == j: o += 1
        occur.append(o)    
    return lookup_array[occur.index(max(occur))]


def INDEX(lookup_array, row_number):
    '''Analogous to the Excel INDEX function'''
    return lookup_array[row_number]
    

def COUNTIF(lookup_array, value):
    '''Analogous to the Excel COUNTIF function'''
    count = 0
    for i in lookup_array:
        if i == value:
            count += 1
    return count


# Not really of interest here but I used this function to sort the sentences by constraint in ascending order 
def REINDEX(array):
    array_ascend = np.sort(array)
    unique, idx = [], []
    for i, x in enumerate(array_ascend):
        if x not in unique:
            idx.append(np.where(array==x)[0].tolist())
            unique.append(x)
    return sum(idx, [])

In [59]:
# A real example from our cloze results

print('Sentence frame = 快考試了，她嘴唇發紫直冒冷汗看起來有點 ___\n')
ans = np.array(['疲憊', '緊張', '緊張', '緊張', '不舒服',
                '嚇人???', '緊張', '不安', '虛弱', '不舒服',
                '糟', '蒼白', '生病', '緊張', '可怕???'])
print("Participants' answers:\n", ans, '\n')

Optimal, occur = INDEX(ans, MODE(MATCH(ans, ans, match_type=0))), COUNTIF(ans, Optimal)

print("Optimal =", Optimal, '(number of occurrences =', occur, ')')
print("Semantic constraint =", occur/len(ans)) 
percentage = [round(COUNTIF(ans, ans[i])/len(ans), 2) for i in range(len(ans))]
print('\nPercentage of each answer (rounded):\n', percentage, '\n')

percentage_dict = {}
for i, x in enumerate(percentage):
    percentage_dict[ans[i]] = x
    
ratings_dict = {'疲憊': 2, '緊張': 3, '不舒服': 2.6, '嚇人???': 3.7, '不安': 1.7,
                '虛弱': 1.5, '糟': 1.4, '蒼白': 2, '生病': 2.4, '可怕???': 3.6}

valence_constraint = sum([ratings_dict[k]*percentage_dict[k] for k in ratings_dict.keys()])
print('Valence constraint = ', valence_constraint)
print('Inexplicit terms = ', 2*0.07 + 3*0.33 + 2.6*0.13 + 3.7*0.07 + 1.7*0.07 + 1.5*0.07 + 1.4*0.07 + 2*0.07 + 2.4*0.07 + 3.6*0.07)

Sentence frame = 快考試了，她嘴唇發紫直冒冷汗看起來有點 ___

Participants' answers:
 ['疲憊' '緊張' '緊張' '緊張' '不舒服' '嚇人???' '緊張' '不安' '虛弱' '不舒服' '糟' '蒼白' '生病' '緊張'
 '可怕???'] 

Optimal = 緊張 (number of occurrences = 5 )
Semantic constraint = 0.3333333333333333

Percentage of each answer (rounded):
 [0.07, 0.33, 0.33, 0.33, 0.13, 0.07, 0.33, 0.07, 0.07, 0.13, 0.07, 0.07, 0.07, 0.33, 0.07] 

Valence constraint =  2.6090000000000004
Inexplicit terms =  2.6090000000000004


In [16]:
percent = {}
for i in range(len(df)):
    ans = [x for x in df.iloc[i, 5:25] if pd.isnull(x) == False]
    percent[i] = [COUNTIF(ans, ans[j])/len(ans) for j in range(len(ans))]
    percent[i].extend([None for j in range(len(percent[i]), 20)])
col_percent = ['Ans%_'+str(i+1) for i in range(20)]
df_percent = pd.DataFrame(percent, index=col_percent).transpose()

display(df_percent)

df_new = pd.concat([df, df_percent], axis=1)
print('\n')
df_new.to_excel('Ans_Percentage.xlsx')

,Ans%_1,Ans%_2,Ans%_3,Ans%_4,Ans%_5,Ans%_6,Ans%_7,Ans%_8,Ans%_9,Ans%_10,Ans%_11,Ans%_12,Ans%_13,Ans%_14,Ans%_15,Ans%_16,Ans%_17,Ans%_18,Ans%_19,Ans%_20
0,0.066667,0.400000,0.400000,0.133333,0.133333,0.200000,0.200000,0.400000,0.133333,0.066667,0.400000,0.133333,0.400000,0.200000,0.400000,NaN,NaN,NaN,NaN,NaN
1,0.133333,0.066667,0.133333,0.066667,0.133333,0.066667,0.066667,0.066667,0.066667,0.200000,0.066667,0.200000,0.200000,0.133333,0.066667,NaN,NaN,NaN,NaN,NaN
2,0.062500,0.062500,0.062500,0.062500,0.187500,0.062500,0.187500,0.125000,0.125000,0.187500,0.062500,0.062500,0.062500,0.125000,0.062500,0.125000,NaN,NaN,NaN,NaN
3,0.133333,0.133333,0.066667,0.066667,0.066667,0.066667,0.133333,0.066667,0.066667,0.133333,0.133333,0.066667,0.133333,0.066667,0.066667,NaN,NaN,NaN,NaN,NaN
4,0.066667,0.133333,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.133333,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,NaN,NaN,NaN,NaN,NaN
5,0.150000,0.750000,0.750000,0.150000,0.750000,0.050000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.150000,0.750000,0.750000,0.750000,0.750000,0.750000,0.05,0.75
6,0.850000,0.850000,0.850000,0.850000,0.850000,0.100000,0.850000,0.850000,0.850000,0.850000,0.850000,0.850000,0.100000,0.850000,0.050000,0.850000,0.850000,0.850000,0.85,0.85
7,0.100000,0.750000,0.750000,0.750000,0.750000,0.750000,0.050000,0.750000,0.750000,0.050000,0.100000,0.750000,0.750000,0.750000,0.050000,0.750000,0.750000,0.750000,0.75,0.75
8,0.133333,0.066667,0.133333,0.133333,0.066667,0.133333,0.066667,0.266667,0.066667,0.266667,0.066667,0.066667,0.066667,0.266667,0.266667,NaN,NaN,NaN,NaN,NaN
9,0.055556,0.055556,0.388889,0.055556,0.388889,0.055556,0.388889,0.388889,0.388889,0.111111,0.111111,0.055556,0.055556,0.388889,0.055556,0.388889,0.055556,0.055556,NaN,NaN


In [ ]:
# you won't need this, this is just for demonstration purposes as we don't yet have the actual rating values
from random import randrange
ratings = [randrange(1,8) for i in best]

dic = {}
for i, x in enumerate(best):
    dic[x] = ratings[i]
    
mapper, idx, val_constr = np.empty(len(df), dtype=object), np.empty(len(df), dtype=object), np.empty(len(df), dtype=object)

for i in range(len(df)):
    mapper[i] = [dic[df.iloc[i, 5:25][j]] for j in range(20) if df.iloc[i, 5:25][j] in dic.keys()]
    s = set(df.iloc[i, 5:25])
    if np.nan in s: s.remove(np.nan)
    idx[i] = [df.iloc[i, 5:25].tolist().index(j) for j in s]
    
for i in range(len(df)):
    val_constr[i] = sum([mapper[i][idx[i][j]]*df_new.iloc[i, 25:45][idx[i][j]] for j in range(len(idx[i]))])

df_new.insert(len(df_new.columns), 'Val_Constraint', val_constr)
display(df_new)
df_new.to_excel('Valence_Constraint.xlsx')